In [1]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error
from sklearn.preprocessing import PolynomialFeatures
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler

In [2]:
# Создаем сэмпл, который определяет стоимость затрат на отопление в минусовой период времени
n_samples = 1000
outside_temperature = np.random.randint(0, 45, n_samples) # ниже нуля в абсолютно значении
inside_temperature = np.random.randint(15, 25, n_samples)
gas_humidity = np.random.randint(10, 90, n_samples)
outside_humidity = np.random.randint(50, 95, n_samples)
inside_humidity = np.random.randint(20, 60, n_samples)
boiler_power = np.random.randint(2, 34, n_samples)
speed_coolant = np.random.randint(1, 4, n_samples)

cost_heating = (outside_temperature 
 * inside_temperature 
 * gas_humidity 
 * outside_humidity 
 / inside_humidity 
 / boiler_power 
 / speed_coolant)

data = (pd.DataFrame({'outside_temperature': outside_temperature,
                      'inside_temperature': inside_temperature, 
                      'gas_humidity': gas_humidity, 
                      'outside_humidity': outside_humidity, 
                      'inside_humidity': inside_humidity, 
                      'boiler_power': boiler_power, 
                      'speed_coolant': speed_coolant,
                      'cost_heating': cost_heating,
                     }))
data.head(5)

,outside_temperature,inside_temperature,gas_humidity,outside_humidity,inside_humidity,boiler_power,speed_coolant,cost_heating
0,12,23,36,68,43,17,2,462.139535
1,32,22,32,62,48,12,1,2424.888889
2,34,17,23,72,56,30,1,569.742857
3,13,22,35,84,56,20,1,750.750000
4,28,19,38,92,26,22,3,1083.841492


In [3]:
# Строим линейную регрессию
X = data[['outside_temperature', 
          'inside_temperature', 
          'gas_humidity', 
          'outside_humidity', 
          'inside_humidity',
          'boiler_power',
          'speed_coolant',
         ]]
y = data['cost_heating']
reg = LinearRegression().fit(X, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(X)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [  122.68823995    86.281834      57.10526986    44.41852298
   -91.84827807  -219.3819453  -1338.80699828]
Bias: 2399.092074944882
Error: 2208.868178746886


In [4]:
# Полученная модель плохо описывает затраты на отопление

In [5]:
# Пробуем вычислить параметры, которые лучше всего влияют на точность модели
def get_better_error_with_different_variables(list_of_x, data, y):
    final_list_of_parameters = []
    better_error = max(y)
    for _ in list_of_x:
        interim_list_of_parameters = []
        interim_list_of_parameters.extend(final_list_of_parameters)
        interim_list_of_parameters.append(_)
        X = data[interim_list_of_parameters]
        reg = LinearRegression().fit(X, y)
        pred_values = reg.predict(X)
        error = mean_absolute_error(pred_values, y)
        if error < better_error:
            better_error = error
            final_list_of_parameters.append(_)
    print(f'Наименьшая ошибка: {better_error},\nконечный список параметров {final_list_of_parameters}')

list_of_x = ['outside_temperature', 'inside_temperature', 'gas_humidity', 'outside_humidity', 'inside_humidity', 'boiler_power', 'speed_coolant']
get_better_error_with_different_variables(list_of_x, data, y)    

Наименьшая ошибка: 2274.0856557858256,
конечный список параметров ['outside_temperature', 'boiler_power', 'speed_coolant']


In [6]:
# Вариант с уточнение списка наиболее значимых параметров также не дал эффект

In [7]:
# Пробуем вариант с использование полиноминальных признаков
X = data[['outside_temperature', 
          'inside_temperature', 
          'gas_humidity', 
          'outside_humidity', 
          'inside_humidity',
          'boiler_power',
          'speed_coolant',
         ]]
y = data['cost_heating']
poly = PolynomialFeatures(2)
X_poly = poly.fit_transform(X)
reg = LinearRegression().fit(X_poly, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(X_poly)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-1.87888433e-10  9.36957266e+01  1.16443065e+02  1.18838837e+02
  5.18454581e+01 -2.10775020e+02 -5.77630578e+02 -1.47994520e+03
  1.14661252e+00  6.78741598e+00  3.05680277e+00  3.38442390e+00
 -5.56146716e+00 -1.11339372e+01 -6.47083246e+01  1.78061623e+00
  1.12255439e+00 -6.42480291e-01  1.12916884e+00 -7.77919625e+00
 -6.75599727e+01  4.31816443e-01  1.24549663e+00 -2.78918042e+00
 -5.48877907e+00 -3.60457837e+01  4.96290838e-01 -1.60980683e+00
 -3.87928617e+00 -3.71509634e+01  2.82462803e+00  8.30605683e+00
  5.74789828e+01  2.05101694e+01  1.24034956e+02  6.56684107e+02]
Bias: 2853.464604510606
Error: 1809.7973777354052


In [8]:
# Меняем степень полиноминального количества
X = data[['outside_temperature', 
          'inside_temperature', 
          'gas_humidity', 
          'outside_humidity', 
          'inside_humidity',
          'boiler_power',
          'speed_coolant',
         ]]
y = data['cost_heating']
poly = PolynomialFeatures(degree=6, include_bias=True, order='C')
X_poly = poly.fit_transform(X)
reg = LinearRegression().fit(X_poly, y)
print('Weights: {}'.format(reg.coef_))
print('Bias: {}'.format(reg.intercept_))
pred_values = reg.predict(X_poly)
print('Error: {}'.format(mean_absolute_error(pred_values, y)))

Weights: [-6.47002490e-11  6.93113674e-07  1.06683478e-07 ... -1.54243893e-04
 -5.00462744e-05 -1.21857349e-05]
Bias: 596.864443737491
Error: 2.2743851749496002e-08


In [9]:
# Использование полиноминальных признаков дает снижение ошибки, пробуем подобрать параметры признаков

In [12]:
best_error = max(y)
for degree in range(1, 11, 1):
    for include_bias in [True, False]:
        for order in ['C', 'F']:
            poly = PolynomialFeatures(degree=degree, include_bias=include_bias, order=order)
            X_poly = poly.fit_transform(X)
            reg = LinearRegression().fit(X_poly, y)
            pred_values = reg.predict(X_poly)
#             print(degree, include_bias, order)
            error = mean_absolute_error(pred_values, y)
            if error < best_error:
                best_error = error
                best_degree = degree
                best_include_bias = include_bias
                best_order = order

print(f'Наименьшая ошибка - {error}\n\
Степень полиноминальных характеристик - {best_degree}\n\
Смещение - {best_include_bias}\n\
Порядок вывода в массиве - {best_order}')

Наименьшая ошибка - 7.108766436836333e-08
Степень полиноминальных характеристик - 8
Смещение - True
Порядок вывода в массиве - F


In [14]:
# Вариант 2
pipe = Pipeline(steps=[('std_slc', StandardScaler()), 
                       ('pca', PCA()),
                       ('poly', PolynomialFeatures()),
                       ('linear_Reg', LinearRegression())])

parameters = dict(pca__n_components=list(range(1,X.shape[1]+1,1)), 
                  poly__degree=list(range(1, 11, 1)), 
                  poly__include_bias=[True, False],
                  poly__order=['C', 'F'])


lf_GS = GridSearchCV(pipe, parameters)
lf_GS.fit(X, y)

degree, include_bias, order = (lf_GS.best_estimator_.get_params()['poly__degree'], 
                      lf_GS.best_estimator_.get_params()['poly__include_bias'], 
                      lf_GS.best_estimator_.get_params()['poly__order'])

print(f'Степень полиноминальных характеристик - {degree}\n\
Смещение - {include_bias}\n\
Порядок вывода в массиве - {order}')

Степень полиноминальных характеристик - 6
Смещение - False
Порядок вывода в массиве - C


In [15]:
poly = PolynomialFeatures(degree=degree, include_bias=include_bias, order=order)
X_poly = poly.fit_transform(X)
reg = LinearRegression().fit(X_poly, y)
pred_values = reg.predict(X_poly)
error = mean_absolute_error(pred_values, y)
print(f'Наименьшая ошибка - {error}')

Наименьшая ошибка - 2.3839985482609905e-08
